In [1]:
import datetime
import numpy as np
import tensorflow as tf
import binarynet_classifier as bc
# import mobilenet_classifier as mc
tf.set_random_seed(777)  # reproducibility

csv_path = tf.train.string_input_producer(['demo2/roll_record/straight_pwm.csv'],
                                         shuffle=False,
                                         name='csv_path')

reader = tf.TextLineReader()
key, value = reader.read(csv_path)

# Read image path and label path
record_defaults = [[0],[""],[0.],[0.]]
_, im_name, label, _ = tf.decode_csv(value, record_defaults=record_defaults)



# load images from image_path above
im_content = tf.read_file(im_name)
image = tf.image.decode_png(im_content, channels=3)
image = tf.cast(image, tf.float32)
image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
image = tf.image.resize_images(image, [64, 64])


'''
The line below binds image and label to the same queue, 
so whenever you carry out an operation on either im_batch or lb_batch, 
the queue will pop out a batch number of data units from the other. 
So a common mistake may be calling im_batch.eval() and lb_batch.eval() 
separately:

# this is wrong
images = im_batch.eval()
labels = lb_batch.eval()
# this is correct
images, labels = sess.run([im_batch, lb_batch])
'''
train_x_batch, train_y_batch = \
    tf.train.batch([image, label], batch_size=250)
train_y_batch = tf.expand_dims(train_y_batch, axis=1)

learning_rate = 0.0001
training_epochs = 100
batch_size = 250
num_classes = 1
num_batch = 4500 // batch_size


X_ph = tf.placeholder(tf.float32, [None, 64, 64, 3])    
# X_ph = tf.placeholder(tf.float32, [batch_size, 160, 320, 3])
# X_grey = tf.image.rgb_to_grayscale(X_ph)
# X_img_resized = tf.image.resize_bilinear(X_grey, [224, 224], align_corners=True)
Y_ph = tf.placeholder(tf.float32, [None, num_classes])

# hypothesis, probs, _ = mc.mobilenet(X_ph, 1)
hypothesis, probs, _ = bc.binarynet(X_ph, 1)
print("hello wrold")
cost = tf.reduce_mean(tf.square(hypothesis - Y_ph))
# print(tf.trainable_variables())  ## Check trainable variables
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
# optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
                      
# initialize
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)


saver = tf.train.Saver()
date_time = str(datetime.date.today()) + '_' + str(datetime.datetime.now().hour) + '_' + str(datetime.datetime.now().minute)


# train my model
print('Learning started.')
x_batch, y_batch = sess.run([train_x_batch, train_y_batch]) # Since, we can't feed tensor
for epoch in range(training_epochs):
    avg_cost = 0
    for step in range(num_batch):
        hy_val, prob, cost_val, _ = sess.run([hypothesis, probs, cost, optimizer], 
                                 feed_dict={X_ph: x_batch, Y_ph: y_batch})
        avg_cost += cost_val / num_batch
        #if step % 10 == 0:
        #    print(step, "Cost: ", cost_val)
    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.9f}'.format(avg_cost))
save_path = saver.save(sess, "saved_networks/" + date_time)
print("Learning Done!")
coord.request_stop()
coord.join(threads)

# # Test model and check accuracy
# correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# print('Accuracy:', sess.run(accuracy, feed_dict={
#     X: test_x_batch, Y:test_y_batch}))

# # Get one and predict
# r = random.randint(0, mnist.test.num_examples -1)
# print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
# print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X:mnist.test.images[r:r + 1]}))



Instructions for updating:
Use the `axis` argument instead
hello wrold
Learning started.
Epoch: 0001 cost = 0.006149091
Epoch: 0002 cost = 0.005087893
Epoch: 0003 cost = 0.005011575
Epoch: 0004 cost = 0.004987668
Epoch: 0005 cost = 0.004983968
Epoch: 0006 cost = 0.004982096
Epoch: 0007 cost = 0.004978836
Epoch: 0008 cost = 0.004974737
Epoch: 0009 cost = 0.004970748
Epoch: 0010 cost = 0.004965635
Epoch: 0011 cost = 0.004953106
Epoch: 0012 cost = 0.004913799
Epoch: 0013 cost = 0.004738454
Epoch: 0014 cost = 0.004330317
Epoch: 0015 cost = 0.003965541
Epoch: 0016 cost = 0.003731031
Epoch: 0017 cost = 0.003560064
Epoch: 0018 cost = 0.003431440
Epoch: 0019 cost = 0.003320999
Epoch: 0020 cost = 0.003216661
Epoch: 0021 cost = 0.003109848
Epoch: 0022 cost = 0.003000186
Epoch: 0023 cost = 0.002890875
Epoch: 0024 cost = 0.002780645
Epoch: 0025 cost = 0.002664427
Epoch: 0026 cost = 0.002546898
Epoch: 0027 cost = 0.002436855
Epoch: 0028 cost = 0.002329631
Epoch: 0029 cost = 0.002218241
Epoch: 0030 

In [ ]:
hy_val[39]